## 깊은 네트워크의 문제점

네트워크가 깊어지게 되면, 그 네트워크가 좋은 결과를 낼 수 있도록 학습을 시키기가 점점 더 어려워지며, 그 원인은 크게 보면 다음 두 가지로 정리할 수 있다.

- Vanishing/Exploding Gradient 문제: CNN에서 파라미터 update를 할 때, gradient값이 너무 큰 값이나 작은 값으로 포화되어 더 이상 움직이지 않아 학습의 효과가 <br /> 
없어지거나 학습 속도가 아주 느려지는 문제가 있다. 네크워크가 깊어질수록 이 문제는 점점 더 심각해지며, 이 문제를 피하기 위해 batch normalization, <br /> 
파라미터의 초기값 설정 방법 개선 등의 기법들이 적용되고 있지만, layer 개수가 일정 수를 넘어가게 되면 여전히 골치거리가 된다.

- 학습 과정의 어려움: 네트워크가 깊어지게 되면, 파라미터의 수가 비례적으로 늘어나게 되어 overfitting의 문제가 아닐지라도 오히려 에러가 커지는 상황이 발생한다.

## ResNet 팀의 실험 - 네트워크가 깊어졌을 경우 학습의 결과

VGGNet을 ImageNet 데이터를 이용해 학습을 시키는 경우 16-레이어와 19-레이어의 결과가 거의 차이가 없는 것을 확인하였다. <br /> 
만약에 19-레이어 이상으로 레이어의 갯수를 증가시킨다면 어떤 결과가 나타났을까? <br />
VGGNet 설계자들이 더 많은 레이어의 수에 대한 실험을 했는지는 확인할 길이 없지만, 갯수를 더 이상 증가를 시키게 되면 결과는 나빠지는 방향으로 나올 것은 분명하다. <br />
ResNet팀은 네트워크가 깊어지는 경우 어떤 결과가 나오는지 실험을 하였다. <br /> 
짧은 시간에 실험의 효과를 얻기 위해 ImageNet보다는 간단한 CIFAR-10 학습 데이터를 20 레이어와 56 레이어에 대하여 비교 실험을 하였으며, 결과는 아래 그림과 같다.
___

<img src="./Images/1.png" width=700 />

실험 결과를 보면, 학습 error rates와 테스트 error rates 모두 56-레이어의 결과가 20-레이어보다 나쁘게 나오는 것을 확인할 수 있으며, <br /> 
다른 학습 데이터에 대하여 레이어의 갯수를 달리하면서 실험을 해봐도 학습 결과가 오히려 더 나빠지는 것을 확인할 수 있었다. <br />
네트워크가 깊어지면 결과가 더 좋아져야 하는데 왜 이런 결과가 나온 것일까? 그 원인은 앞서 언급한 두 가지 문제 때문이다.

## Residual Learning

ResNet 설계 팀은 네트워크를 100 레이어 이상으로 깊게 하면서, 깊이에 따른 학습 효과를 얻을 수 있는 방법을 고민하였으며, 그 방법으로 Residual Learning이라는 <br /> 
방법을 발표하였다. Residual이라는 단어를 사용한 이유는 아래 구조를 살펴보면 파악할 수가 있다.
___

<img src="./Images/2.png" width=400 />

먼저 위 그림의 구조와 같은 평범한 CNN 네트워크를 살펴보자. 이 평범한 네트워크는 입력 x를 받아 2개의 weighted layer를 거쳐 출력 $H(x)$를 내며, 학습을 통해 최적의 $H(x)$를 <br /> 
얻는 것이 목표이며, Weight layer의 파라미터 값은 그렇게 결정이 되어야 한다.
___

그런데 위 네트워크에서 $H(x)$를 얻는 것이 목표가 아니라 $H(x) - x$를 얻는 것으로 목표를 수정한다면, 즉 출력과 입력의 차를 얻을 수 있도록 학습을 하게 된다면, <br /> 
2개의 weighted layer는 $H(x) – x$를 얻도록 학습이 되어야 한다. 여기서 $F(x)=H(x)-x$라면, 결과적으로 출력 $H(x)$는 $H(x)=F(x)+x$ 가 된다. <br />
그러므로 위 그림은 아래 그림처럼 바뀌며, 이것이 바로 Residual Learning의 기본 구조가 된다. <br /> 
달라진 점이 있다면, 입력에서 바로 출력으로 연결되는 shortcut 연결이 생기게 되었으며, 이 shortcut은 파라미터가 없이 바로 연결이 되는 구조이기 때문에 연산량 관점에서는 <br /> 
덧셈이 추가되는 것 외에는 차이가 없다.
___

<img src="./Images/3.png" width=500 />
___

이렇게 관점을 조금 바꾸었을 뿐이지만, 꽤 많은 효과를 얻을 수 있다. 예전에는 $H(x)$를 얻기 위한 학습을 했다면, 이제는 $H(x) – x$를 얻기 위한 학습을 하게 되며, <br /> 
최적의 경우라면 $F(x)$는 0이 되어야 하기 때문에 학습할 방향이 미리 결정이 되어, 이것이 pre-conditioning 구실을 하게 된다. <br /> 
$F(x)$가 거의 0이 되는 방향으로 학습을 하게 되면 입력의 작은 움직임(fluctuation)을 쉽게 검출할 수 있게 된다. <br /> 
그런 의미에서 $F(x)$가 작은 움직임, 즉 나머지(residual)를 학습한다는 관점에서 residual learning이라고 불리게 된다. <br />
또한 입력과 같은 $x$가 그대로 출력에 연결이 되기 때문에 파라미터의 수에 영향이 없으며, 덧셈이 늘어나는 것을 제외하면 shortcut 연결을 통한 연산량 증가는 없다. <br />
그리고 몇 개의 layer를 건너 뛰면서 입력과 출력이 연결이 되기 때문에 forward나 backward path가 단순해지는 효과를 얻을 수 있다.

결과적으로 identity shortcut 연결을 통해 얻을 수 있는 효과는 다음과 같다.
- 깊은 네트워크도 쉽게 최적화가 가능하다.
- 늘어난 깊이로 인해 정확도를 개선할 수 있다.

Residual Learning 이란 개념이 고안된 이유는 VGGNet과 같은 기존 방식으로는 어느 일정 이상의 레이어 수를 넘어서게 되면, 결과가 나빠지는 문제를 해결하기 위함이다. <br />
실제로 ResNet 팀은 2015년 ILSVRC에 출전하여 classification, localization 및 detection 분야에서 모두 우승을 차지했으며, classification에서는 152-레이어를 사용하여 <br /> 
top-5 error율이 3.57%로, 훈련을 받은 사람이 이미지를 분류할 때의 오차율(약 5%정도)보다 더 좋은 결과를 얻었으며, 아래 그림과 같다.
___


<img src="./Images/4.png" width=600 />

2014년 우승을 차지한 GoogLeNet보다 약 두 배 성능이 좋았으며, 네트워크의 깊이도 7배 이상으로 깊어졌다. <br /> 
GoogLeNet 팀은 이후 계속 자신들의 CNN 구조를 개선하여 2016년 2월 Inception-V4/Inception-ResNet까지 발표를 하였으며, ResNet보다 더 좋은 결과를 얻었다.

## ResNet 팀의 실험

ResNet 팀은 실험을 위한 네트워크를 설계하면서 VGGNet의 설계 철학을 많이 이용하였다. 그래서 대부분의 Conv 레이어는 3x3 kernel을 갖도록 하였으며,  다음 2가지 원칙을 지켰다. <br /> 
- 출력 feature map 크기가 같은 경우, 해당 모든 레이어는 모두 동일한 수의 filter를 갖는다.
- Feature-map의 크기가 절반으로 작아지는 경우는 연산량의 균형을 맞추기 위해 filter의 갯수를 두 배로 늘린다.  <br />
Feature-map의 크기를 줄일 때는 pooling을 사용하는 대신에 convolution을 수행할 때, stride의 크기를 “2”로 하는 방식을 취한다. <br />
또한 복잡도(연산량)를 줄이기 위해 max pooling(1곳 제외), hidden FC, drop out 등을 사용하지 않았다.

이들은 비교를 위해 평범한 네트워크(plain network)와 residual 네트워크로 구별을 하였다. <br /> 
Plain network의 경우도 VGGNet 보다 filter의 갯수를 줄이고 복잡도를 낮춤으로써 34-레이어의 plain network는 19-layer의 VGGNet에 비해 <br /> 
연산량을 20% 미만이 되도록 하였다. (VGGNet 19-layer: 19.6 billion FLOPs, ResNet 팀이 설계한 34-layer Plain setwork: 3.6 billion FLOPs) <br />
Residual 네트워크도 구조를 간단하게 하기 위해, 매 2개의 Conv 레이어마다 shortcut connection이 연결되도록 하였다. <br />
이들이 실험에 사용한 네트워크의 구조는 다음 그림과 같다. (그림은 대표적으로 34-레이어의 구조만을 보여준다)
___

<img src="./Images/5.png" width=500 />
___

또한 실험에는 18-레이어, 34-레이어, 50-레이어, 101-레이어 및 152-레이어에 대하여 수행을 하였으며, 각각에 대한 레이어 구성은 다음 표와 같다.
___

<img src="./Images/6.png" width=800 />

위 표를 보면 18-레이어와 34-레이어는 동일한 구조를 사용하였고, 다만 각 레이어에 있는 Conv 레이어의 갯수만 다르다는 것을 알 수가 있다. <br />
하지만 50-/101-/152- layer의 경우는 18-34-layer와 구조가 다르다는 것을 알 수가 있는데, 그 이유는 뒤에서 살펴볼 예정이다.
___

## 실험 결과

18-레이어와 34-레이어에 대한 Plain network과 Residual network의 실험 결과를 살펴보면 다음 그림과 같다.
___

<img src="./Images/7.png" width=800 />

Plain network의 경우는 34-레이어의 결과가 18-레이어의 결과보다 약간 나쁘다는 것을 알 수가 있으며, Residual network은  <br />
34-레이어가 18-레이어의 결과보다 좋은 것을 알 수가 있다. <br /> 
___

아래 표는 각각의 경우에 대한 top-1 error rates를 비교한 결과이다.
___

<img src="./Images/8.png" width=500 />

또 한가지 주목할 점은 학습의 초기 단계에서 residual 네트워크의 수렴 속도가 plain 네트워크보다 빠르다는 점이다.
___

위 실험 결과를 종합해보면, residual 네트워크가 plain 네트워크에 비해 더 좋은 결과를 내면서도 빠르다는 것을 알 수 있다.

## Deeper Bottleneck Architecture

학습에 걸리는 시간을 고려하여 50- /101-/152- 레이어에서는 기본 구조를 조금 변경을 시켰으며, residual function은 1x1, 3x3, 1x1으로 아래 그림처럼 구성이 된다. <br /> 
___

<img src="./Images/9.png" width=600 />

Bottleneck 구조라고 이름을 붙인 이유는 차원을 줄였다가 뒤에서 차원을 늘리는 모습이 병목처럼 보이기 때문이라고 생각된다.
___

이렇게 구성한 이유는 연산 시간을 줄이기 위함이다. 먼저 맨 처음 1x1 convolution은 NIN(Network-in-Network)이나 GoogLeNet의 Inception 구조에서 살펴본 것처럼 <br /> 
dimension을 줄이기 위한 목적이며, 이렇게 dimension을 줄인 뒤 3x3 convolution을 수행 한 후, 마지막 1x1 convolution은 다시 dimension을 확대시키는 역할을 한다. <br /> 
결과적으로 3x3 convolution 2개를 곧바로 연결시킨 구조에 비해 연산량을 절감시킬 수 있게 된다.

## Deeper layer (50-/101-/152-레이어) 실험 결과

Single model에 대한 실험 결과는 아래 표와 같다.
___

<img src="./Images/10.png" width=500 />

152-레이어에 대한 top-5 error rates는 4.49% 수준까지 떨어졌다. Single model 만으로도 과거에 발표된 어떤 구조보다도 좋은 결과를 얻을 수 있다.
___

최종적으로 ILSVRC에 제출한 결과는 아래와 같다. 결과를 제출할 당시는 2개의 152-레이어의 결과를 조합해서 얻은 결과이며,(ensemble) <br /> 
최종적으로는 depth를 달리하는 6개 모델에 대한 ensemble로 사용하였다.
___

<img src="./Images/11.png" width=500 />

ResNet은 기본적으로 VGGNet과 유사한 구조를 사용했지만, 연산량의 균형을 맞추기 위해 VGGNet과는 다른 모델로 발전을 하게 되었으며, Residual 네트워크의 구조를 사용하여 <br /> 
152-레이어라는 아주 깊은 네트워크를 이용해 아주 뛰어난 결과를 얻었고, 이렇게 깊은 네트워크에서도 충분히 학습이 가능하다는 것을 확인 할 수 있다.

## CIFAR-10 데이터에 대한 실험

CIFAR-10 이미지 데이터 집합은 32x32 크기의 작은 이미즈 데이터들의 집합이며, 학습 이미지 5만장과 test 이미지 1만장으로 구성되어 있다. <br /> 
총 10 개의 class가 있으며, 각각의 class에는 학습과 검사 이미지를 합하여 총 6000장이 있다. <br />
CIFAR-10 이미지 데이터 는 유명한 토론토 대학교의 Alex Krizhevsky, Vinod Niar 및 Geoffrey Hinton이 수집을 하였으며, 이미지의 크기가 작아 컴퓨터 비전이나 <br />
머신 러닝의 초기 알고리즘이나 구조 검증에 많이 사용이 된다. 10개의 class에 대한 예는 다음 그림과 같다.
___

<img src="./Images/12.png" width=500 />
___

ResNet 팀은 CIFAR-10에 대한 검증을 위해 네트워크의 구성을 약간 변형을 시켰다. 그 이유는 ImageNet(ILSVRC) 데이타의 경우는 224x224로 이미지의 크기가 컸지만, <br /> 
CIFAR-10에 있는 이미지는 32x32로 크기가 작기 때문이다. 검증 방식은 동일한 레이어의 수를 갖는 Plain 네트워크와 Residual 네트워크를 비교 실험하는 것이다. <br />
ImageNet 데이터를 이용한 실험에서는 맨 처음 Conv 레이어에 7x7 filter를 사용했지만 CIFAR-10의 이미지는 크기가 매우 작기 때문에, <br />
맨 처음 Conv 레이어에 3x3 filter를 사용하였다. 다음은 6n개의 3x3 Conv 레이어 그룹이 온다. <br /> 
6n 중 각각의 2n에 대하여 feature map의 크기가 {32, 16, 8}이 되도록 하였으며, filter의 개수는 연산량의 균형을 맞춰주기 위해 {16, 32, 64}가 되도록 하였다. <br />
맨 마지막에 global average pooling과 10-way softmax를 배치하였으며, 결과적으로 전체 layer의 수는 “6n + 2”가 되었다.

전체적인 블럭의 구성은 아래 그림과 같다. 
___

<img src="./Images/13.png" width=400 />

위 그림은 개념적 이해를 돕기 위해 간단하게 14-레이어를 갖는 구조만을 보여주는 것이며, 레이어의 갯수가 많아지면 각각의 2n 그룹에 레이어를 추가하면 된다.
___

실제로 ResNet 팀은 실험 시 n값을 바꿔가면서 더 깊은 layer에서 어떤 결과가 나타나는지 비교 실험을 하였다.

n = {3, 5, 7, 9}로 설정을 하면 20- /32- /44- /56-레이어를 얻을 수 있으며, 이들에 대한 실험 결과는 다음 그림과 같다. 
___

<img src="./Images/14.png" width=700 />

ImageNet 데이터를 이용한 실험과 거의 동일한 양상의 결과를 보여준다.
___

Plain 네트워크는 일정 레이어의 갯수를 넘어서면 레이어의 수가 증가함에 따라 결과가 더 나빠지는 경향성을 보이지만, Residual 네트워크는 레이어의 수가 증가하더라도 <br /> 
결과가 더 좋아짐을 알 수가 있다. 특히 Residual 망에 대해서는 n = 18로 설정하여  101-레이어까지 실험을 하였는데 56-레이어 때에 비해 성능이 더 좋아지는 것을 확인할 수 있다.

## 레이어 별 Response에 대한 분석

Plain 네트워크와 Residual 네트워크에 대하여 각각의 레이어에서의 response에 대한 표준 편차를 살펴보았다. <br /> 
이 결과는 Batch Normalization은 적용 하고 ReLU 및 addition은 적용하기 전을 비교한 것이며, 결과는 아래 그림과 같다.
___

<img src="./Images/15.png" width=500 />

위 그림 중 밑에 있는 그림은 표준편차를 크기 별로 다시 sorting한 결과이다.
___

위 결과를 보면, Residual 네트워크의 response가 Plain 네트워크에 비해 작다는 것을 확인할 수 있다. <br />
Residual 네트워크의 결과가 Plain 네트워크의 결과에 비해서 표준편차가 작으며, 이 실험 결과를 통해 네트워크의 response가 크게 흔들리지 않는 것을 확인할 수 있었으며, <br /> 
결론적으로 이런 이유로 인해 네트워크가 깊어졌을 때 Plain 네트워크에서 발생하는 문제가 Residual 네트워크에서는 적게 발생하게 된다.

## 1000 layer 넘었을 때 결과

추가 실험으로, n = 200으로 설정하여 1202-레이어의 깊이를 갖는 Residual 네트워크에 대한 실험을 해봤는데, 110-레이어의 경우보다 결과가 약간 나쁘게 나왔다. <br /> 
___

<img src="./Images/16.png" width=250 />

하지만 1000-레이어가 넘는 경우에도 최적화에 별 어려움이 없었음을 언급하였다.
___

1000-레이어가 넘었을 때, 결과가 더 나쁘게 나온 이유는 네트워크의 깊이에 비해 학습 데이터의 양이 부족했고, 결과적으로 학습 데이터에만 특화된, 즉 과적합된 결과가 <br /> 
나왔을 것으로 추정한다. 이 실험에서는 maxout이나 drop out과 같은 별도의 regularization 기술은 적용을 하지 않았으며, 단지 네트워크를 깊게 했을 때 어떤 결과가 <br /> 
나오는지를 확인하기 위한 목적이었이며, 추가적인 regularization 기법을 적용하면 이것보다는 결과를 개선시킬 수 있을 것이라 생각된다.

그리고 이것은 다음 해 Kaiming He 가 발표한 논문(Identity Mappings in Deep Residual Networks)을 보면. Pre-activation을 통해 개선이 되는 것을 확인할 수 있다. <br /> 
이 논문에서는 2015년 발표한 Residual Network의 구조를 개선한 새로운 구조를 소개하였다.

## 다른 실험 결과 (2016년)

Kaiming He가 2016년 논문을 통해 발표한 실험 데이터를 보면, CIFAR-10뿐만 아니라 CIFAR-100에 대해서도 비슷한 실험을 하였으며, 그 결과는 아래 표와 같다.
___

<img src="./Images/17.png" width=500 />

<img src="./Images/18.png" width=400 />

이 실험은 단지 경향성을 확인하기 위한 실험이며, 그렇기 때문에 적당한 수준의 data augmentation만 적용 했음을 밝히고 있다.
___

1001-레이어에서는 pre-activation을 통해 기존 Residual 네트워크ㅡ의 이슈가 개선되었다는 사실을 확인할 수 있다.

Image localization이나 detection 에서도 ResNet을 사용하는 것이 과연 효과가 있을까? <br />
결론부터 이야기 하면, 아래의 표처럼 2015년 ILSVRC에서 localization과 detection 모두 2위와 상당한 격차를 보이며 우승한 것으로 볼 때, ResNet이 효과가 있음을 알 수 있다.
___

<img src="./Images/19.png" width=600 />
___

ResNet에서는 image detection/localization의 성능을 위해, Faster R-CNN 방법을 적용한다. <br /> 
Faster R-CNN을 이해하려면, R-CNN, SPPNet, Fast R-CNN 및 Faster R-CNN으로 이어지는 detection 분야의 계보를 이해해야 한다.

R-CNN은 2013년 당시 버클리에 다니고 있던 Ross Girshick등이 발표했으며, CNN을 image detection 분야에 적용하여, SIFT나 HOG와 같은 gradient 기반의 <br /> 
computer vision 알고리즘을 적용한 것보다 훨씬 좋은 결과를 거두었다. <br /> 
또한 이후 연구자들이 image detection 분야에 CNN을 적용한 연구에 불을 붙이게 되는 도화선 역할을 한다. <br />
R-CNN은 성능은 뛰어나지만 속도가 느린 문제가 있다. R-CNN의 느린 속도 문제를 개선한 SPPNet은 2014 Kaiming He(ResNet 발표자) 등이 발표를 했으며, <br />
Spatial Pyramid Pooling이라는 독특한 개념을 적용하여 속도를 개선시킨다. <br />
이후 2015년 Ross Girshick이 마이크로소프트로 자리를 옮겨 R-CNN과 SPPNet의 문제를 개선한 Fast R-CNN을 발표한다. <br /> 
이후 Shaoqing Ren, Kaiming He 및 Ross Girshick이 공동 저자로 더 개선시킨 Faster R-CNN을 발표하였으며, <br /> 
논문의 부제도 “Towards Real-time Object Detection with Region Proposal Networks”로 붙이고, 실시간 객체 검출을 목표로 하였다. <br /> 이 Faster R-CNN의 개념이 다시 ResNet으로 이어져 localization/detection 분야에서 성과를 보이게 된다.

## R-CNN과 SPPNet의 문제점

R-CNN이 그 이전 알고리즘보다 뛰어난 성능을 보이기는 했지만, 다음과 같은 3가지 문제점을 갖고 있다. <br />

- Training이 여러 단계로 이루어짐. <br />
R-CNN은 크게 3단계 과정으로 이루어진다. 우선 약 2000여개의 후보 영역에 대하여 log loss 방식을 사용하여 fine tuning을 한다. <br /> 
이후 ConvNet 특징을 이용하여 SVM에 대한 fitting 작업을 진행한다. <br /> 
끝으로 bounding box regressor(검출된 객체의 영역을 알맞은 크기의 사각형 영역으로 표시하고 위치까지 파악)에 대한 학습을 한다.

- Training 시간이 길고 대용량 저장 공간이 필요. <br />
SVM과 bounding box regressor의 학습을 위해, 이미지의 후보 영역으로부터 feature를 추출하고 그것을 디스크에 저장한다. <br /> 
PASCAL VOC07 학습 데이터 5,000장에 대하여 2.5일 정도가 걸리며, 저장 공간도 수백 GigaByte를 필요로 한다.

- 객체 검출(object detection) 속도가 느림. <br />
학습이 오래 걸리는 문제도 있지만, 실제 검출할 때도 875MHz로 Overclocking된 K40 GPU에서 이미지 1장을 처리하는데 47초가 걸린다. <br />

R-CNN의 개념을 잘 표현한 Girshick의 slide 그림은 아래와 같다.
___

<img src="./Images/20.png" width=500 />

앞서 살펴본 것처럼 후보 영역에 대하여 모든 연산을 순차적으로 수행하는 구조이기 때문에 느릴 수밖에 없다.
___

R-CNN의 가장 큰 문제는 모든 객체 후보 영역에 대하여 개별적으로 연산을 한다는 점이며 이것 때문에 속도가 매우 느리다. <br /> 
SPPNet 설계자들은 Spatial Pyramid Pooling을 사용하여 convolution 연산을 공유할 수 있는 방법을 제안하였다. <br /> 
이를 통해 R-CNN보다 학습에서는 3배 정도 빠르고, 실제 적용 시 약 10~100배 정도 빠른 결과가 나왔다. <br />
하지만 SPPNet은 구조적인 관점에서는 R-CNN과 마찬가지로 학습에 3 단계 파이프 라인을 적용하는 것은 동일하다. 그리고 이렇게 얻어진 feature들은 디스크에 저장되어야 한다. <br /> 
또한 R-CNN과 달리 fine-tuning 알고리즘을 적용할 때 Spatial Pyramid Pooling 앞단에 있는 Conv 레이어에 대해서는 fine tuning을 하지 않는데, 이로 인해 <br /> 
deep network를 적용할 때는 정확도가 문제가 될 수도 있다. Girshitck의 slide에서 SPPNet을 한 눈에 설명한 구조는 아래 그림과 같다.
___

<img src="./Images/21.png" width=600 />

SPPNet의 ConvNet 단계는 전체 이미지에 대하여 한꺼번에 연산을 하고 그 결과를 공유하고, SPP layer를 거치면서 region 단위 연산을 수행한다.
___

## Fast R-CNN

Fast R-CNN은 기본적으로 검출 정확도(mAP)가 R-CNN이나 SPPNet보다는 좋으며, 학습 시 multi-stage가 아니라 single-stage로 가능하고, 학습의 결과를 <br /> 
네트워크에 있는 모든 레이어에 update할 수 있어야 하며, feature caching을 위해 별도의 디스크 공간이 필요 없는 방법을 개발하는 것을 목표로 삼았다. <br />
Fast R-CNN의 기본 구조는 다음 그림과 같다. 
___

<img src="./Images/22.png" width=700 />

Fast R-CNN 네트워크는 전체 이미지 및 객체 후보 영역을 한꺼번에 받아들인다.
___

Convolution과 max pooling을 통해 이미지 전체를 한번에 처리를 하고 feature map을 생성한다. 그 후 각 객체 후보 영역에 대하여 ‘RoI Pooling layer’를 통해, <br /> 
feature-map으로부터 fixed-length feature 벡터를 추출한다. 이 부분은 SPPNet의 Spatial Pyramid Pooling과 하는 일이 유사하다고 볼 수 있다. <br />
이렇게 추출된 fixed-length feature vector는 FC 레이어에 인가되며, 뒷 단에는 “object class + background”를 추정하기 위한 softmax부분과 <br /> 
각각의 object class의 위치를 출력하는 bbox(bounding box) regressor가 온다.

Fast R-CNN의 구조를 앞서 그린 것과 비슷한 방식으로 표현을 하면 아래 그림과 같다.
___

<img src="./Images/23.png" width=550 />

먼저 test time 시에 구조적으로 SPPNet과 비슷하게 전체 이미지에 대해 ConvNet 연산을 1번만 수행을 하고 그 결과를 공유하며, RoI Pooling 레이어에서 다양한 후보 영역들에 대하여 FC 레이어로 들어갈 수 있도록 크기를 조정해주는 과정을 거친다. <br /> 
이후 단계는 Softmax classifier와 Bounding Box regressor를 동시에 수행하기 때문에 multi-stage pipeline 과정을 순차적으로 수행하는 SPPNet에 비해 빠르다.
___

Fast R-CNN의 training 과정을 보여주는 다음 그림을 보면, SPPNet이나 기존 R-CNN과 달리 일반 CNN처럼 모든 layer에 대하여 학습이 가능하며 전체를 1 stage로 <br /> 
학습이 가능하다. 
___

<img src="./Images/25.png" width=500 />

SPPNet에서 ConvNet 부분은 1단계에서 학습을 마치면 파라미터가 고정이 되고, 단지 마지막 3-레이어(FCs + SVM/BBox Regressor)만 학습이 가능했었다. <br /> 
하지만 Fast R-CNN은 1-stage 연산을 하기 때문에 ConvNet 부분까지 error를 backpropagation 시키면서 학습이 가능해졌고, 이를 통해 정확도에 대한 개선이 가능해졌다.
___

## Fast R-CNN에서 학습 시간을 줄인 비법

R-CNN과 SPPNet은 학습 시에 mini-batch 크기를 128개로 정했고, 128개의 서로 다른 이미지로부터 128개의 RoI를 취하는 region-wise sampling 방법을 사용하였으며, <br /> 
아래 그림과 같다.
___

<img src="./Images/26.png" width=700 />

이 방법을 Fast R-CNN에 동일하게 적용하면 속도가 느려질 가능성이 있다. R-CNN은 모든 후보 영역 RoI를 224x224 크기로 scale 하여 사용하지만, <br /> Fast R-CNN에서는 원 이미지를 그대로 사용하기 때문에 경우에 따라서는 RoI 영역에 해당하는 receptive field의 크기가 매우 클 수 있으며, <br />
최악의 경우는 전체 이미지가 될 수도 있다. 그렇게 되면 이미지의 크기에 비례하여 연산량이 증가하여 느려질 수 있게 된다.
___

이 문제를 피하기 위해 mini-batch를 구성할 때 hierarchical sampling 방식을 사용하였다. R-CNN, SPPNet처럼 서로 다른 128장의 학습 이미지에서 RoI를 <br /> 
무작위로 선택하는 것이 아니라, 작은 수의 학습 이미지(논문에서는 2장으로 정함)으로부터 128개의 RoI를 정하도록 하였으며, 아래 그림과 같다.
___

<img src="./Images/27.png" width=700 />

이렇게 하면, test time과 마찬가지로 training time에도 학습 이미지의 결과를 공유할 수 있게 되어 연산 속도가 빨라지게 된다.
___

## Fast R-CNN 성능

다음 그래프는 Girshick의 slide에서 따온 것이며, 2013년 CNN 기법을 적용한 알고리즘이 적용되면서 object detection 성능이 비약적으로 발전하는 것을 볼 수 있으며, <br /> 
저자는 자신의 slide에서 “object detection renaissance”라는 표현을 하였다.
___

<img src="./Images/28.png" width=700 />

아래 그림에서 볼 수 있는 것처럼, R-CNN 기법을 적용하면서 mAP가 PASCAL VOC 이미지 데이터에서 50%를 넘어가지 시작했으며, Fast R-CNN을 적용하였을 때 <br />
70% 수준에 이르게 된다.
___

R-CNN, SPPNet, Fast R-CNN의 비교 실험 결과는 다음과 같다. (여기서 R-CNN과 SPPNet은 동일한 환경에서의 실험을 위해, VGGNet-16에 구조에 올려 실험한 결과이다.)
___

<img src="./Images/29.png" width=700 />
___

Fast R-CNN은 R-CNN과 SPPNet이 갖고 있던 object detection/localization알고리즘의 정확성과 속도 문제를 개선하였다. 

R-CNN과 Fast R-CNN을 속도 관점에서만 살펴보면 아래 표와 같다.
___


<img src="./Images/30.png" width=600 />

이 표를 보면, Fast R-CNN은 Test time에 1장의 이미지를 처리하는데 0.32s로 상당히 빨라진 것처럼 보인다. <br /> 
그런데 이 시간에는 object가 있을 것이라고 생각되는 후보 영역을 계산하기 위한 region proposal 시간이 고려되지 않았다. <br />
그러므로 실제적으로 region proposal 시간까지 고려하면, 이미지 1장을 처리하는데는 약 2초 정도가 걸리게 되며, 실시간 관점에서 보면 2초는 부담스러운 시간이 된다.
___

위 표에서 살펴본 것처럼, Fast R-CNN이 Test time 처리시간을 R-CNN보다 많이 개선시키기는 하였지만, 여전히 object detection을 위한 후보 영역을 계산하는 <br /> 
과정을 필요로 하고, 이 과정에서 상당한 시간이 걸리는 점은 여전히 문제이다. <br /> 
또한 region proposal이 별도의 과정으로 되어 있기 때문에 진정한 의미에서 1 pass로 구현했다고 볼 수 없다.

그리하여 Faster R-CNN 설계자들은 region proposal을 위해 별도의 과정을 거치는 대신에 image classification을 위한 ConvNet에 region proposal을 <br /> 
위한 특수한 용도의 망(RPN)을 추가하여 Fast R-CNN의 문제를 해결하였다.

Shaoqing Ren, Kaiming He 및 Ross Girshick이 공동 저자로 object detection/localization 성능을 대폭 개선한 논문 <br />
“Faster R-CNN: Towards Real-time Object Detection with Region Proposal Networks”를 중심으로 Faster R-CNN에 대하여 살펴보자.

## R-CNN/Fast R-CNN과 Region Proposal

Fast R-CNN을 동작을 다시 보면, 입력 이미지와 함께, 적절한 region proposal 방법을 통한 region proposal을 같이 받아들이며, <br />
이것은 RoI Pooling layer를 거쳐 FC 레이어로 넘어가고 최종 단에는 Softmax classifier 와 bounding box regressor가 온다.
___

<img src="./Images/31.png" width=600 />
___

R-CNN 때부터 Region proposal 방법으로 적용했던 방식은 “selective search” 방식이며, 이 과정은 아래 그림과 같다.
___

<img src="./Images/32.png" width=700 />

즉 bottom-up segmentation을 수행하고 이것들을 다양한 scale 상에서 병합을 하고 이 region을 사각형 box 영역으로 구분하는 것이다.
___

2015년 Jan Hosang 등이 발표한 논문 “What makes for effective detection proposals?”에는 저자가 기존에 발표된 detection 관련 proposal에 <br /> 
대한 비교 분석한 자료가 있으며, 그 결과는 다음 표와 같다.
___

<img src="./Images/33.png" width=800 />

이 결과를 보면 EdgeBox 방식이 가장 뛰어나다는 것을 확인할 수 있으며, selective search 방식도 전체적인 평가에서 좋은 편에 속하지만, 연산 시간 관점에서 보면 <br />
결과 좋은 방식이라고 볼 수는 없다.

## RPN(Region Proposal Network)

Faster R-CNN의 블락도는 아래 그림과 같다.
___

<img src="./Images/34.png" width=400 />

Fast R-CNN의 기본 구조와 비슷하지만, Region Proposal Network(RPN)이라고 불리는 특수한 네트워크가 추가되었으며, 이 RPN을 이용하여 object가 있을만한 <br />
영역에 대한 proposal을 구하고 그 결과를 RoI pooling layer에 보낸다. RoI pooling 이후 과정은 Fast R-CNN과 동일하다.
___

이 RPN은 Fast R-CNN에서 사용했던 동일한 ConvNet을 그대로 사용하기 때문에 입력의 크기에 제한이 없으며, 출력은 각 proposal에 대하여 “objectness score”가 붙은 <br /> 
사각형 object의 집합이 되며, model의 형태는 fully-convolutional network 형태다. <br />
이 RPN은 Conv Net 부분의 최종 feature map을 입력으로 받아들인 후, $n$x$n$ 크기의 sliding window convolution을 수행하여 256 차원 혹은 512차원의 벡터를 <br /> 
만들어내고 이 벡터를 다시 물체인지 물체가 아닌지를 나타내는 box classification (cls) 레이어와 후보 영역의 좌표를 만들어 내는 box regressor (reg) 레이어에 연결한다. <br />
k개의 object 후보에 대하여 cls layer 에서는 object인지 혹은 object가 아닌지를 나타내는 score를 구하기 때문에 2k score가 되고, reg layer에서는 각각의 object에 대하여 <br /> 
4개의 좌표 $(X, Y, W, H)$값을 출력하기 때문에 4k 좌표가 된다. <br />
전체적인 RPN에 대한 구성은 아래 그림과 같다.
___

<img src="./Images/35.png" width=400 />

각각의 sliding window에서는 총 k개의 object 후보를 추천할 수 있으며, 이것들은 sliding window의 중심을 기준으로 scale과 aspect ratio를 달리하는 조합 <br />(
논문에서는 anchor라고 부름)이 가능하다. 논문에서는 scale 3가지와 aspect ratio 3가지를 지원하여, 총 9개의 조합이 가능하다.
___

아래의 그림은 PASCAL VOC 2007에 있는 영상에 대하여 RPN을 이용하여 proposal을 낸 예제들이다.
___

<img src="./Images/36.png" width=500 />

이렇게 sliding window 방식을 사용하게 되면, anchor와 anchor에 대하여 proposal을 계산하는 함수가 “translation-invariant” 하게 된다는 것이다. <br /> 
이것은 CNN에서 sliding window를 사용하여 convolution을 하였을 때 얻는 효과와 동일하다.
___

이 translation-invariant한 성질로 인해 model의 수가 크게 줄어들게 된다. $k=9$인 경우에 (4+2)x9 차원으로 차원이 줄어들게 되어, 결과적으로 연산량을 크게 절감할 수 있다.

## Faster R-CNN 결과

Faster R-CNN에 대한 학습 결과는 아래 표와 같다.
___

<img src="./Images/37.png" width=600 />

위 표에서 SS는 Selective Search 방식을 의미하고, EB는 EdgeBox 방식을 의미한다.
___

먼저 연산 시간을 줄이기 위해 ZFNet을 이용한 test를 진행하였으며, SS와 EB는 별도로 region proposal을 만든 후 detection을 진행한 것이고, “RPN + ZF, shared”는 <br /> 
RPN과 ZFNet을 사용한 Fast R-CNN 블럭의 경우로 region proposal이 300개 정도였음에도 불구하고 정확도가 1% 이상 좋다는 것을 확인할 수 있다. <br />
결과적으로 보면 region proposal을 별도로 수행하는 것보다 RPN을 이용하여 동시에 수행을 하더라도 결과가 더 나쁘지 않다.

다음은 연산 속도 관점에서의 결과를 살펴보면 아래 표와 같다.
___

<img src="./Images/38.png" width=700 />

Selective search를 이용한 region proposal을 수행하고 Fast R-CNN을 하는 경우는 약 2초 정도가 소요가 되었지만, RPN을 이용해 region proposal을 수행하고 <br />
Fast R-CNN의 VGG-16 모델과 결합시키는 경우는 약 0.2 초가 걸린다. <br /> 
결과적으로 초당 5fps 속도로 object detection이 가능하기 때문에 real-time 영역에 근접하게 되었다.
___

또한 아래 표처럼, Fast R-CNN 블락으로 비교적 망이 단순한 ZFNet을 사용한다면, 약 17fps까지 속도가 올라가게 된다.
___

<img src="./Images/39.png" width=700 />
___

ResNet의 object detection/localization의 기본 알고리즘은 Faster R-CNN에 기반하고 있다.

Object detection은 크게 "feature extraction"부분과 "object classifier(classifier+bounding box regressor)부분으로 나누어 생각할 수 있다. <br />
기존에는 object detection의 성능을 향상시키기 위해 HOG나 SIFT와 같은 그라디언트 기반의 Computer vision 알고리즘을 이용하여 feature를 추출하고, <br />
SVM이나 DPM과 같은 classifier를 적용하여 object를 검출하였다.

그러다가 2013년 R-CNN이 object detection에 CNN을 적용하면서, object detection 성능이 크게 개선이 되었다. <br />
ResNet 개발팀은 object detection 분야에도 정통한 사람들이었으며, CNN/Deep CNN을 이용하여 object detection 성능을 끌어올리는 기반을 제공하였다. <br />
이는 R-CNN, SPPNet, Fast R-CNN 및 Faster R-CNN의 기반이 되었다. ResNet은 classfication 성능 뿐만 아니라 detection/localization에서도 <br />
2015년 ILSVRC에서 우승을 차지한다.

## Faster R-CNN과 RPN(Region Proposal Network)

Faster R-CNN은 기본적으로 Fast R-CNN과 동일하지만, 결정적인 차이는 object의 후보 영역을 추천하기 위한 region proposal 과정이 별도의 과정으로 <br /> 
되어 있는 것이 아니라, 아래 그림과 같이 Faster R-CNN 네트워크에 후보 영역을 추천하기 위한 RPN(Region Proposal Network)이 포함이 되어 있다는 점이다. <br />
___

<img src="./Images/40.png" width=300 />
___

## NoC(Network on Conv feature map)

Faster R-CNN을 구현한 논문을 보면, ZFNet과 VGG-16을 이용하여 Faster R-CNN을 구현하였다. 마지막 Conv Net에서 얻어진 feature-map을 <br />
RPN(Region Proposal Network)이라고 불리는 특수한 네트워크의 입력으로 보내고, 이 RPN을 이용하여 object가 있을만한 영역에 대한 proposal을 구한 후 <br /> 
RoI pooling layer에 보낸다.

RestNet의 구조는 VGGNet과는 구조가 다른데 어떻게 Faster R-CNN을 구현할까? <br />
Faster R-CNN에 있는 Conv Net 부분은 ResNet으로 구현하는데 큰 무리가 없을 것 같은데, RPN과 RoI Pooling Layer 및 뒷 단에 있는 classifier와 <br /> 
bounding box regressor 부분은 어떻게 구현을 해야 할까?

ResNet 개발팀은 ResNet은 구조적으로 Faster R-CNN이나 Fast R-CNN의 블럭 그림과 모양이 다른데, ResNet 구조를 크게 변화시키지 않고 Faster R-CNN을 <br /> 
구현하기 위한 고민을 하였다, 그 고민에 대한 해답은 그들이 발표한 논문 "Object Detection Networks on Convolutional Feature Maps"라는 논문에 답이 있다.

이 논문에 따르면, object detection은 크게 보면 feature extraction 부분과 object classifier 부분으로 나눌 수 있다. <br />
CNN을 이용한 feature 추출 방법이 기존에 발표된 어떤 computer vision 알고리즘을 사용하는 것보다 훨씬 성능이 뛰어나다는 점은 이미 밝혀졌고, <br />
이들은 object classifier의 성능에 영향을 주는 것이 무엇인지를 이 논문에서 밝히고 있다.
___

<img src="./Images/41.png" width=600 />

위 그림은 이 논문에 나오는 블럭 그림 이며, feature extractor 부분은 Conv Net으로 구현하기 때문에 ResNet으로 충분히 구현이 가능하다는 것을 알 수 있다. <br />
이후 Region Proposal 및 RoI Pooling을 담당하는 부분도 구조적인 관점에서 보았을 때 ConvNet으로 구현이 가능하다.
___

Object classifier 부분은 CNN을 통해서 얻어진 feature를 기반으로 별도의 네트워크를 통해서 처리한다는 관점에서 "NoC(Network on Conv feature map)"이라고 <br /> 
명명하였으며, NoC 부분을 어떻게 구성을 하면 정확도가 개선이 되는지를 논문에서 밝히고 있다.

Classifier 부분에 대한 전형적인 구현 방법은 MLP(Multi-Layer Perceptron)를 fc(FC 레이어) 2~3개로 구현을 하는 방법이다. <br />
하지만 이 논문에서는 기존처럼 단순하게 fc(FC 레이어)로 구현을 하는 것이 아니라 이부분을 하나의 네트워크로 간주를 하고 어떨 때 최적의 성능이 나오는지를 <br />
실험으로 확인을 하였다.

정확도를 개선하려면, 기존처럼 FC 레이어만을 사용하는 것보다는 FC 레이어 앞에 Conv 레이어를 추가하면 결과가 좋아지며, multi-scale에 대응하기 위해 인접한 <br />
scale에서의 feature map 결과를 선택하는 "Maxout" 방식을 지원하면 결과가 더 개선되는 것을 확인할 수 있다. <br />
아래 그림은 Maxout을 적용한 경우를 보여주는 그림이다.
___

<img src="./Images/42.png" width=500 />
___

## ResNet에서 Faster R-CNN을 구현하기

Faster R-CNN을 ResNet에서 구현하기 위해 그들은 101-layer 구조를 이용하였으며, NoC 개념처럼 RoI Pooling 및 그 이후 부분을 Conv Net으로 구현하도록 하였다.<br />
 다양한 layer 수에 따른 ResNet 구성의 경우를 살펴보면 아래 표와 같다.
___

<img src="./Images/71.png" width=700 />

Faster R-CNN을 VGG-16에 구현할 때와 마찬가지로 ResNet을 다시 구성을 해서 살펴보면, 위 ResNet 구조를 "feature extractor + object classifier"로 구분할 수 있다.
___

Feature extractor 부분은 conv1, conv2_x, conv3_x, conv4_x(101 layer ResNet에서 91 layer 부분에 해당)를 이용하여 구현할 수 있으며, 
이렇게 되면, <br /> 
VGG16 때와 마찬가지로 feature map에서의 1 픽셀은 stride 16에 해당이 된다. <br />
또한 이 layer들은 RPN(Region Proposal Network)과 Fast R-CNN Network 공유하여 사용하게 된다.

RPN 이후 과정은 Fast R-CNN 구조(아래 그림에서 회색 영역에 해당)를 구현을 하여야 한다. <br /> 
이미 Con1 ~ Conv4는 feature를 추출하기 위한 용도로 사용을 했으니, 당연히 Conv5 부분과 Classifier 부분을 잘 활용하거나 변화를 시켜야 한다는 것은 감을 잡을 수 있다.
___

<img src="./Images/43.png" width=600 />

ConvNet을 거쳐 얻어진 feature map을 conv5_1에 보내기 전에 RoI pooling을 수행하며, Conv5_x 부분은 위 그림에서 FCs에 해당하는 부분을 수행을 하며, <br />
최종단의 Classifier는 위 그림의 최종단으로 대체를 해주면, ResNet을 약간만 변형을 하면, object detection에 활용이 가능하다.
___

## ResNet을 이용한 object detection 결과

ResNet을 사용하여 위와 같은 방식으로 object detection을 수행한 결과, VGG-16을 사용하여 object detection을 수행한 것보다 정확도가 아래 표처럼 <br /> 
개선되는 것을 확인할 수 있다.
___

<img src="./Images/44.png" width=500 />

모두 VGG-16을 사용할 때보다 ResNet을 사용하는 것이 결과가 좋아지는 것을 확인할 수 있다. 이는 ResNet을 이용한 더 깊은 네트워크의 효과를 본 결과라고 볼 수 있다.
___

ResNet이 2015년 처음 발표되었을 당시, ResNet 팀은 residual network이라는 독특한 개념을 적용한다는 관점에서는 혁신적이었고 원래 구조만으로도 충분한 성능을 보였지만, <br />
‘ILSVRC-2015에 참가’라는 시간상의 제한으로 인해서 구조적인 고려가 충분하지 못하였다. <br /> 
ResNet의 원래 구조를 설명하는 논문 “Deep Residual Learning for Image Recognition”을 자세히 살펴보면 몇 곳에서 그런 부분을 찾을 수 있다. <br />
ResNet 팀은 2015년 대회를 마치고 자신들의 아키텍처에 대하여 심층적인 고민과 실험을 한 것으로 보이며, 그 결과 2016년에 <br />
“Identity Mappings in Deep Residual Networks” 라는 논문을 통해 “pre-activation” 이라는 개념을 적용한 개선된 Residual Network 구조를 발표하게 된다.



## Residual Network에 대한 고찰

기존 Residual Network의 기본 구조는 아래 그림과 같으며 입력과 출력의 관계는 그림에 있는 수식과 같다.
___


<img src="./Images/45.png" width=600 />

여기서 ${x}_{l+1}$은 identity $함수 h({x}_{l})$와 Residual 함수 $F({x}_{l})$의 합에 대하여 activation 함수인 ReLU를 적용한 결과이다.<br /> 
그래서 “after-add mapping” 이라는 이름이 붙은 것이다.
___

위 그림처럼 기존 ResNet에서는 함수 $h({x}_{l})$를 identity mapping을 위한 short-cut connection이라고 불렀지만, 실제로는 identity 함수와 residual 함수의 <br /> 
합을 더한 후 ReLU를 수행하기 때문에 진정한 의미에서의 identity라고 부르기는 힘들다. <br />
Residual Network 1개만을 고려했을 때는 큰 차이가 없지만, 이것들을 여러 개를 연결시킨다고 생각을 해보면, addition 뒤에 오는 ReLU로 인해 blocking이 <br /> 일어나기 때문에 ${x}_{l+1}$이 ${x}_{l}$과  $F(x)$의 합으로 표현되는 것이 아니라, 위 그림의 식처럼 함수 $f$ 처리를 해줘야 한다. <br /> 
(합이 아니라 곱의 형태가 됨)

만약에 $f$를 identity 함수라고 한다면 위 그림은 아래와 같이 바뀔 수 있다. 위 그림과 비슷해 보이지만 $f$가 identity 함수 이기 때문에 수식이 아래 그림처럼 바뀌게 된다.
___

<img src="./Images/46.png" width=600 />

그럼 위 그림에서 볼 수 있는 수식의 변화는 무엇을 의미하는 것일까? <br />
위 수식의 진정한 의미는 임의의 위치에서의 ${x}_{L}$을 ${x}_{l}$와 residual 함수의 합으로 표시할 수 있다는 것이며, 이것은 forward나 backward path를 <br />
아주 간단하게 표현할 수 있게 만든다. 즉, 특정 위치의 출력은 특정 위치에서의 입력과 residual 함수의 합으로 표현이 가능해진다는 점이다. <br />
전형적인 ConvNet의 경우 forward나 backward path가 성격상 곱(multiplication)의 형태로 표현되는 것과 비교하면 연산이 매우 단순해지는 효과를 얻을 수 있게 된다.
___

<img src="./Images/47.png" width=500 />

그러므로 이렇게 original Residual Network를 개선해주면, 원래의 ResNet의 철학을 거의 유지하면서도 수학적인 관점에서 좀 더 유용하게 된다. <br />
ResNet 연구팀은 이점에 주목을 하여 original ResNet을 개선한다.
___

아래 그림은 forward/backward path를 한눈에 보여준다. 그림에 같이 표시된 수식을 보면 forward/backward path 연산이 단순 덧셈의 형태로 간단하게 구현이 가능해진다.
___

<img src="./Images/48.png" width=500 />

## Identity skip connection의 중요도

Identity skip connection이 무슨 의미를 갖는지 살펴보자. <br />
Short-cut connection이 어떤 경우에 최적의 결과를 도출할 수 있는지 확인을 하기 위해, ResNet팀은 short-cut connection을 다양한 경우로 바꿔가면서 실험하였다.
___

<img src="./Images/49.png" width=600 />

위 그림의 (a)는 original ResNet에서 사용한 구조로 short-cut connection에 다른 어떤 것을 추가하지 않고 identity connection으로 연결시킨 경우이고, <br />
(b)부터 (f)까지는 short-cut connection을 다양하게 변화를 시켜가며 실험을 한 것이다. <br />
110-layer의 ResNet에 대하여 CIFAR-10데이터를 이용하여 실험을 수행한 결과 원래의 identity shortcut의 결과가 가장 좋다는 것을 확인할 수 있었다. <br /> 
다양한 경우에 따른 에러율은 위 그림에 표시가 되어 있다.
___

(b)부터 (f)까지는 short-cut connection이 multiplication에서 blocking된 형태이기 때문에 앞서 확인한 것처럼 정보가 곧바로 전달이 되는 것이 아니라, <br /> 
곱셈에 의해 변형된 형태로 전달이 되며, 이로 인해 최적화 문제가 생긴다. 결과적으로 identity skip connection이 속도나 성능 면에서 최적이라는 것을 확인할 수 있다.

## Activation 함수 위치에 따른 성능 체크

ResNet 팀은 activation 함수의 위치가 어떤 영향을 주는지 확인을 하기 위해 다음 그림과 같은 다양한 조합에 대하여 실험을 하였다.
___

<img src="./Images/50.png" width=600 />

이 그림에서 (a)는 ResNet의 원래 버전이며, addition 뒤에 ReLU가 오는 형태 (after-add)이다. (b)부터 (e)까지는 다양한 경우에 어떤 결과가 나오는지 확인하기 위한 조합이다.
___

(b)는 기본 ResNet 구조에서 BN(Batch Normalization)의 위치를 addition 뒤로 옮긴 경우이며 그 결과는 아래 그림과 같다.
___

<img src="./Images/51.png" width=500 />

결과적으로 BN이 propagation path를 방해할 수 있으며, 위 그림에서 볼 수 있는 것처럼, training과 test에서 모두 original 보다 결과가 좋지 못한 것을 확인할 수 있다.
___

(c)는 addition 뒤쪽에 있는 ReLU를 residual net쪽으로 옮긴 경우이다. 원래는 residual 함수가 (-∞, +∞) 범위로 나오는데 ReLU를 residual net 쪽으로 <br /> 
옮기면 residual net의 결과가 항상 non-negative 결과가 나오게 되며, 이로 인해서 네트워크의 representation 능력이 떨어지고 결과적으로 성능이 나빠진다. <br />
(d)와 (c)는 pre-activation 성능을 확인하기 위한 구조이다. Pre-activation 을 고려하는 이유는 원래의 residual net에 있는 ReLU가 진정한 <br /> 
identity mapping 개념을 방해하기 때문에 이것을 움직일 수 있는지 확인하기 위함이다.

아래 그림을 살펴보자. 그림 (a)는 원래 ResNet 구조인데 여기서 act. layer의 위치를 조정하면 (b)와 같은 형태를 만들어 낼 수 있다. 
___

<img src="./Images/52.png" width=600 />

원래 ResNet의 identity mapping 함수를 asymmetric하게 조정하면, activation의 위치를 residual 쪽으로 옮길 수 있으며, 이것을 다시 정리하면 (c)와 같은 형태가 된다.
___

Activation의 위치가 weight layer 앞쪽으로 오면서 pre-activation 구조를 갖게 된다. <br />
이 pre-activation 구조에서 BN layer가 ReLU 앞으로 간 형태를 “full pre-activation”이라고 부르며, 앞에 오는 BN에 의해서 regularization 효과를 <br />
얻게 되어 결과가 좋아지게 되며, 아래 그림은 실험 결과이다.
___

<img src="./Images/53.png" width=400 />

Pre-activation을 사용하면 Test  error가 original ResNet보다 좋아졌다는 것을 확인할 수 있지만, Training에서는 iteration의 횟수가 커지게 되면 <br />
original보다 약간 결과가 나쁜 것을 확인할 수 있다. 이것은 regularization을 통해 일반화가 더 잘되어서 이런 결과가 나온 것으로 추정이 된다.
___

즉 original은 학습 데이터에 더 특화된 결과를 보이지만 일반화(generalization) 특성은 개선된 ResNet보다 좋지 못하기 때문에 test 결과가 나빠지는 것으로 볼 수 있다.

## 성능 비교

순수하게 activation 위치에 따른 성능만 비교하기 위해, single-crop (320x320) ImageNet 데이터에 대한 실험 결과는 아래 표와 같다.
___

<img src="./Images/54.png" width=600 />

네트워크가 깊어질수록 pre-activation 을 사용한 ResNet이 original ResNet보다 결과가 좋아진다는 것을 확인할 수 있다.
___

아래 표는 다양한 네트워크 구조에 따른 실험 결과이다 역시 pre-activation을 사용한 ResNet이 가장 좋은 결과를 내는 것을 확인할 수 있다. 
___

<img src="./Images/55.png" width=600 />
___

개선된 ResNet의 구조에 대한실험 결과를 종합해보면, short-cut connection의 구조는 간단할수록 결과가 좋으며 identity mapping을 사용하는 것이 가장 좋았다. <br /> Original ResNet에서는 ReLU의 위치가 addition 뒤에 위치되며, 결과를 forward/backward propagation 시킬 때 약간의 blocking 구실을 했었는데, 수학적인 조정을 통해 <br /> 
activation의 위치를 Residual net쪽에 위치를 시키고 그것도 weight layer 앞쪽에 위치를 시키면, 수학적으로 완결된 형태가 되고 이로 인해서 결과가 더욱 개선이 된다. <br />
또한 개선된 구조에서는 forward/backward propagation이 쉬워지기 때문에 1000-layer가 넘는 deep network를 어렵지 않게 구현할 수 있다.


GoogLeNet 설계 팀은 2014년 인셉션 구조(Inception-V1)을 발표한 이래 계속 계손된 Inception 구조를 발표한다.

2014년 ILSVRC에서 영국의 옥스퍼드 대학교팀(VGGNet)은 GoogLeNet과 근소한 차이로 2위를 차지하지만, VGGNet은 그 구조의 간결함으로 인해 이후 많은 딥러닝 팀들이 <br />
VGGNet 구조를 사용한다.

GoogLeNet 팀 역시 같은 해 발표된 VGGNet의 개념을 발전된 방식으로 반영하여 convolution kernel에 대한 factorization(인수분해) 개념을 집어 넣어 네트워크를 <br />
더욱 깊게 만들었으며, 이것은 Inception-V2 및 V3에 반영이 된다. <br /> 
Inception-V2는 기본적으로 Inception-V1에 BN(batch normalization)을 반영한 기본 모델을 말하며, 여기에 구조 실험을 위해 convolution factorization, label smoothing 및 <br /> 
Auxiliary classifier에 BN까지 모든 개념을 결합시켜 가면 성능을 꾸준히 발전을 시켰으며, 최종적으로 모든 개념이 적용된 모델을 Inception-V3라고 명명하였다. <br />
Inception-V3구조로는 2015년 ILSVRC 대회에 참가하지는 않았지만, 그 해 우승을 한 ResNet 성능에 근접하는 성능을 낸다. <br /> 
논문에 따르면 multi-model&multi-crop ensemble에서 top-5 에러율이 3.58% 수준에 도달한다다. (논문: Rethinking the Inception Architecture for Computer Vision).
___

<img src="./Images/56.png" width=600 />

2015년에 마이크로소프트팀이 발표한 ResNet 구조는 여러 면에서 혁신적인 구조라서 GoogLeNet 팀도 많은 검토를 하였다.
___

그들이 2016년에 발표한 논문 “Inception-V4, Inception-ResNet and the Impact of Residual Connections on Learning”에 따르면, <br />
일부는 ResNet 팀의 아이디어에 긍정을 하지만, 일부는 꼭 그렇지 않다고 언급하고 있다. <br /> 
자신들이 개선시킨 Inception-V4 구조를 통해서도 ResNet 구조를 사용하지 않더라도 ultra-scale 딥러닝이 가능함을 밝힌다. <br /> 
또한 자신들의 Inception 구조에 ResNet 개념을 접목시킨 Inception-ResNet도 같이 발표하고, Inception 구조에 ResNet을 개념을 접목시킨 새로운 Inception 구조가 <br /> 
ResNet 팀이 원래 발표한 성능보다 좋다는 것도 보여준다.

## Inception-V4

Inception –V4의 기본 구조는 stem 이라고 불리는 9개의 layer를 갖는 부분을 통해 크기가 299x299x3 이미지를 받아 35x35x384 크기의 feature-map을 만들어 낸다. <br /> 
이후 여러 단의 Inception-A, Inception-B 및 Inception-C가 오며, 인셉션 모듈 중간에는 feature map의 크기를 줄이기 위한 Reduction-A와 Reduction-B가 <br /> 
오고 최종단에는 average pooling, dropout 및 softmax layer가 온다. <br /> 
전체적인 구조는 아래 그림과 같다.
___

<img src="./Images/57.png" width=500 />

기본 구조는 이전 Inception 구조와 크게 다르지 않다는 것을 확인할 수 있지만, stem 부분은 이전 인셉션 구조가 비교적 단순한 convolution과 pooling의 조합이었던 반면에, 
Inception-V4는 좀더 복잡한 구조가 되었다는 것을 확인할 수 있다.
___

특히 feature-map의 크기를 줄이는 부분이 이전 인셉션 구조와 달리 형태가 다양한 형태를 취하고 있다.
___

<img src="./Images/58.png" width=500 />
___

Inception-V3와 마찬가지로 35x35 크기의 feature-map으로부터 8x8 크기의 feature-map을 만드는 과정에서는 feature-map에 크기에 따라 3가지 다른 형태의 <br /> 
Inception 모듈을 사용했으며, 다음 그림과 같다.
___

<img src="./Images/59.png" width=600 />
___

Inception-V4에는 이전과 달리 이미지 크기를 줄이기 위한 reduction module이 2개 추가가 되었으며, 이것들의 구조는 아래 그림과 같다.
___

<img src="./Images/60.png" width=600 />
___

## Inception-ResNet

Inception-ResNet은 V1과 V2가 있다. 기본적인 구조는 모두 동일하고, stem 부분에 적용된 layer 구조가 차이가 있다. <br /> 
V1의 stem은 비교적 단순한 구조를 취하고 있지만, V2의 stem은 Inception-V4와 동일한 구조를 취하는 점이 다르다. <br />
또한 ResNet-V2가 ResNet-V1 보다 필터의 개수가 더 많다. <br />
원래의 ResNet과 달리 GoogLeNet 팀은 인셉션 구조의 특성을 고려하여 ResNet 구조를 아래 그림처럼 조금 변형을 시켰다.
___

<img src="./Images/61.png" width=600 />

위 그림처럼 변형을 시킨 이유는 인셉션은 맨 앞단에 오는 1x1 convolution을 통해 차원을 감소시켜 연산량을 줄이는 구조를 취하지만, <br /> 
ResNet 구조에 맞게 feature-map의 차원을 맞춰주면서 연산량을 적정선으로 유지하기 위해 맨 앞단에 오히려 차원을 맞춰주기 위한 1x1 convolution이 온다. 
___

Inception-ResNet의 메인 구조는 아래 그림과 같다. 
___

<img src="./Images/62.png" width=600 />

Stem 부분은 Inception-V4와 동일하다. 전체적으로 Inception-V4나 Inception-Resnet-V2의 메인 부분은 75 layer로 layer 깊이는 동일하다.
___

Inception-ResNet-A, B, C 부분의 구조는 아래 그림과 같다.
___

<img src="./Images/64.png" width=600 />

<img src="./Images/65.png" width=600 />

원래의 ResNet은 맨 앞에 오는 1x1 convolution 부분이 차원을 줄여주는 역할을 했지만, Inception-ResNet은 차원을 유지시켜주기 위해 1x1 convolution을 통해 <br />
만들어내는 feature-map의 개수가 큰 것을 확인할 수 있다.
___

Reduction-A와 Reduction-B의 구조는 다음 그림과 같다.
___

<img src="./Images/66.png" width=600 />
___

## 성능

ILSVRC-2012 데이터를 이용하여, single-crop single-model로 실험한 결과 성능은 아래 표와 같다.
___

<img src="./Images/67.png" width=600 />

BN-Inception은 Inception-V2에 해당한다고 볼 수 있으며, 인셉션 구조가 발전함에 따라 에러율이 개선되는 것을 확인할 수 있다. <br />
Inception-V4와 Inception-ResNet-V2는 거의 비슷한 성능을 보이고 있다.
___

또한 144-crop single-model에 대한 실험도 하였는데, 그 결과는 아래 표와 같다.
___

<img src="./Images/68.png" width=600 />

이 표에서 ResNet-151 결과는 ResNet 팀이 논문에 발표한 성능이다. 이것을 보면 Inception-V3 이상의 구조에서 모두 ResNet-151 보다 성능이 좋다는 것을 알 수 있다.
___

또한 더욱 성능을 끌어올리기 위해 여러 모델에 대한 ensemble을 하여 실험한 결과는 아래 표와 같다.
___

<img src="./Images/69.png" width=600 />

최고의 성능은 Inception-V4와 3개의 Inception-ResNet-V2 모델을 조합했을 때이며, 이 때 top-5 에러율은 3.08%까지 떨어져 현재까지 발표된 모든 딥러닝 논문 중 <br /> 
최고의 성능을 보였다.
___

ResNet 구조를 채용했을 때의 가장 큰 장점 중 하나는 학습속도가 빠르다는 점이며 실험결과는 아래의 그래프를 통해 확인이 가능하다.
___

<img src="./Images/70.png" width=600 />

훨씬 적은 epoch에서 학습 결과가 나오는 것을 확인할 수 있으며, 이는 초기에 최적의 hyper-parameter를 결정할 때 유용하게 활용할 수 있다.
___

기존 인셉션 구조를 개선한 Inception-V4의 성능과 인셉션 구조에 residual net 개념을 도입한 Inception-ResNet-V2는 모두 뛰어난 성능을 보인다. <br /> 
GoogLeNet 팀들이 발표한 인셉션 구조도 뛰어나고, ResNet 팀이 발표한 ResNet 역시 뛰어난 구조임이 확인되었지만, 아직도 sub-network의 구조를 하는 것이 <br /> 
가장 적합한지는 알 수 없다.